In [1]:
import tensorflow as tf
from tensorflow import keras
import keras_tuner as kt

In [2]:
(img_train,label_train),(img_test, label_test) = keras.datasets.fashion_mnist.load_data()

4422102/4422102 [==============================] - 1s 0us/step


In [3]:
img_train = img_train.astype('float32')/255.0
img_test = img_test.astype('float32')/255.0

In [4]:
#definicja modelu
#model typu hipermodel(model stosowany do hipertuningu)
#hipermodel -> 1 korzystanie z funkcju konstruktora modeli
#-> poprzez tworzenie pochodnych klasy HyperModel

def model_builder(hp):
    model = keras.Sequential()
    model.add(keras.layers.Flatten(input_shape=(28,28)))
    #dostrojenie gęstości warstwy -> liczby neuronów
    hp_units = hp.Int('units',min_value=32, max_value=512, step=32)
    model.add(keras.layers.Dense(units=hp_units, activation='relu'))
    model.add(keras.layers.Dense(10))
    #dostrojenie algorytmu optymalizującego (learning_rate)
    hp_learning_rate = hp.Choice('learnig_rate',values=[1e-2,1e-3,1e-4])
    model.compile(optimizer=keras.optimizers.Adam(learning_rate=hp_learning_rate),
                  loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                  metrics=['accuracy'])
    return model


In [5]:
#Utworzenie wystąpienie Tunera i HyperTuning
tuner = kt.Hyperband(model_builder,
                     objective='val_accuracy',
                     max_epochs=10,
                     factor=3,
                     directory='my_dir',
                     project_name='into_to_kt')

In [8]:
#wywołanie zwrotne, wstrzymanie treningu w przypadku istotnej utraty wartości walidacji
stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_loss',patience = 5)

In [9]:
#wyszukiwanie hiperparametrów
tuner.search(img_train, label_train, epochs=50, validation_split = 0.2, callbacks=[stop_early])

best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]
print(f"""
Wyszukiwanie hiperparametrów zakończone. 
Optymalna wartość unitów dla pierwszej epoki: {best_hps.get('units')},
optymalna wartość learning_rate dla optymalizatora wynosi: {best_hps.get('learning_rate')}.
""")

Trial 30 Complete [00h 03m 23s]
val_accuracy: 0.8630833625793457

Best val_accuracy So Far: 0.893583357334137
Total elapsed time: 00h 28m 07s
INFO:tensorflow:Oracle triggered exit


KeyError: KeyError: 'learning_rate does not exist.'

In [10]:
#Trenowanie modelu
model = tuner.hypermodel.build(best_hps)
history = model.fit(img_train,label_train,epochs=50,validation_split=0.2)

val_acc_per_epoch = history.history['val_accuracy']
best_epoch = val_acc_per_epoch.index(max(val_acc_per_epoch))+1
print('Najlepsza epoka: %d' %(best_epoch,))

Epoch 34/50
1500/1500 [==============================] - 32s 21ms/step - loss: 0.1063 - accuracy: 0.9603 - val_loss: 0.4042 - val_accuracy: 0.9000
Epoch 35/50
1500/1500 [==============================] - 32s 21ms/step - loss: 0.1047 - accuracy: 0.9609 - val_loss: 0.4462 - val_accuracy: 0.8895
Epoch 36/50
1500/1500 [==============================] - 32s 21ms/step - loss: 0.1004 - accuracy: 0.9620 - val_loss: 0.4573 - val_accuracy: 0.8954
Epoch 37/50
1500/1500 [==============================] - 32s 21ms/step - loss: 0.0983 - accuracy: 0.9635 - val_loss: 0.4285 - val_accuracy: 0.9000
Epoch 38/50
1500/1500 [==============================] - 32s 21ms/step - loss: 0.0995 - accuracy: 0.9627 - val_loss: 0.4488 - val_accuracy: 0.8960
Epoch 39/50
1500/1500 [==============================] - 32s 21ms/step - loss: 0.0945 - accuracy: 0.9647 - val_loss: 0.4829 - val_accuracy: 0.8913
Epoch 40/50
1500/1500 [==============================] - 14s 9ms/step - loss: 0.0907 - accuracy: 0.9650 - val_loss: 0.

In [11]:
hypermodel = tuner.hypermodel.build(best_hps)
hypermodel.fit(img_train,label_train,epochs=best_epoch,validation_split=0.2)

Epoch 1/34
1500/1500 [==============================] - 7s 4ms/step - loss: 0.4975 - accuracy: 0.8227 - val_loss: 0.3836 - val_accuracy: 0.8618
Epoch 2/34
1500/1500 [==============================] - 6s 4ms/step - loss: 0.3716 - accuracy: 0.8656 - val_loss: 0.3560 - val_accuracy: 0.8727
Epoch 3/34
1500/1500 [==============================] - 6s 4ms/step - loss: 0.3317 - accuracy: 0.8784 - val_loss: 0.3956 - val_accuracy: 0.8518
Epoch 4/34
1500/1500 [==============================] - 6s 4ms/step - loss: 0.3050 - accuracy: 0.8851 - val_loss: 0.3437 - val_accuracy: 0.8771
Epoch 5/34
1500/1500 [==============================] - 6s 4ms/step - loss: 0.2872 - accuracy: 0.8951 - val_loss: 0.3192 - val_accuracy: 0.8835
Epoch 6/34
1500/1500 [==============================] - 6s 4ms/step - loss: 0.2712 - accuracy: 0.8986 - val_loss: 0.3335 - val_accuracy: 0.8773
Epoch 7/34
1500/1500 [==============================] - 6s 4ms/step - loss: 0.2568 - accuracy: 0.9042 - val_loss: 0.3158 - val_accuracy:

In [12]:
eval_result = hypermodel.evaluate(img_test,label_test)
print(f"[test loss, test accuracy]:{eval_result}")

313/313 [==============================] - 1s 2ms/step - loss: 0.4911 - accuracy: 0.8856
[test loss, test accuracy]:[0.49106571078300476, 0.8855999708175659]
